<a href="https://colab.research.google.com/github/sorayutmild/Unsupervised-Thai-Document-Clustering-with-Sanook-news/blob/main/simCSE_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimCSE:Simple Contrastive Learning of Sentence Embeddings Finetuning

## Prepare dataset

In [1]:
!pip -q install pythainlp==3.0.0
!pip -q install python-crfsuite

     |████████████████████████████████| 11.5 MB 6.2 MB/s 
     |████████████████████████████████| 965 kB 6.8 MB/s 


In [2]:
import pandas as pd
import numpy as np
import glob
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
import re
import os
from tqdm.auto import tqdm

In [3]:
text_file_folder = 'text_files'

In [4]:
# https://drive.google.com/file/d/16IhVdoLuFso28TVIpYoIogco_LZg1e17/view?usp=sharing
!gdown --id 16IhVdoLuFso28TVIpYoIogco_LZg1e17

# https://drive.google.com/file/d/1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6/view?usp=sharing
!gdown --id 1chetbLnMLRIqt0U8m3JMzO0xguGUEHX6

!unzip -q ./text_files.zip

Streaming output truncated to the last 5000 lines.
text_files/สุขภาพ_43.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_43.txt),
         continuing with "central" filename version
text_files/สุขภาพ_430.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_430.txt),
         continuing with "central" filename version
text_files/สุขภาพ_431.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_431.txt),
         continuing with "central" filename version
text_files/สุขภาพ_432.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_432.txt),
         continuing with "central" filename version
text_files/สุขภาพ_433.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_433.txt),
         continuing with "central" filename version
text_files/สุขภาพ_434.txt:  mismatching "local" filename (text_files/р╕кр╕╕р╕Вр╕ар╕▓р╕Ю_434.txt),
         continuing with "central" filename version
text_files/สุขภาพ_435.txt:  mismatching "local" fil

In [5]:
df = pd.read_csv('sanook_news_all.csv')
df = df.drop_duplicates('Link').reset_index()
display(df.head())
df.info()

,index,Title,Link,Date,Views,File_name,Label
0,0,คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาที...,https://www.sanook.com/news/7722298/,5 เม.ย. 62,"1,925,802",การเมือง_1.txt,การเมือง
1,1,คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต.ค.,https://www.sanook.com/news/1148178/,16 ต.ค. 55,"1,074,542",การเมือง_2.txt,การเมือง
2,2,อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร,https://www.sanook.com/news/1596593/,22 พ.ค. 57,"788,755",การเมือง_3.txt,การเมือง
3,3,ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก,https://www.sanook.com/news/1773902/,2 เม.ย. 58,"736,582",การเมือง_4.txt,การเมือง
4,4,ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังปร...,https://www.sanook.com/news/7723326/,25 มี.ค. 62,"706,607",การเมือง_5.txt,การเมือง


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14858 entries, 0 to 14857
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      14858 non-null  int64 
 1   Title      14858 non-null  object
 2   Link       14858 non-null  object
 3   Date       14858 non-null  object
 4   Views      14858 non-null  object
 5   File_name  14858 non-null  object
 6   Label      14858 non-null  object
dtypes: int64(1), object(6)
memory usage: 812.7+ KB


In [6]:
def read_text_file(text_path):
    with open(text_path) as f:
        texts = [line.strip() for line in f.readlines()]
    return texts

def get_raw_text(text_list:list):
    '''
    text_list : list of text paragraphs
    return : raw text
    '''
    return '\n'.join(text_list)

def get_list_text(text):
    '''
    get list text, separate with paragraph
    '''
    return text.split('\n')

def include_title(title, text_list):
    return [title] + text_list

def tokenize(text):
    tokenized_text = [word for word in word_tokenize(text) if word and not re.search(pattern=r"\s+", string=word)] # segment and not ' word '
    return tokenized_text

def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_special_char(text):
    special_char = r"""[~@&#<>,}{()*:;"'-+=_?"\"/$%^%$!ๆ‘’“”…่่่่่่่่่ạễ•′–0-9]"""
    text = re.sub(special_char, '', text)
    text = text.replace("[", '').replace("]", '').replace('-', '')
    return text

def clean_text(text):
    '''
    1. link
    2. symbols, numbetrs, special characters
    3. stop words
    '''
    #pre rules
    text = text.lower().strip()
    text = replace_url(text)
    text = replace_special_char(text)
    cleaned_text = text

    return cleaned_text

df['File_path'] = [os.path.join(text_file_folder, fn) for fn in df['File_name'].values]
df['list_text'] = df['File_path'].map(read_text_file)
df['list_text'] = [include_title(t, lt) for t, lt in zip(df['Title'].values, df['list_text'].values)]
df['raw_text'] = df['list_text'].map(get_raw_text)
df['cleaned_text'] = df['raw_text'].map(clean_text)
df['list_clean_text'] = df['cleaned_text'].map(get_list_text)

# clear empty document (only have title)
df = df[df["raw_text"]!=df["Title"]]
df.reset_index(inplace=True, drop=True)

df.head()

,index,Title,Link,Date,Views,File_name,Label,File_path,list_text,raw_text,cleaned_text,list_clean_text
0,0,คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาที...,https://www.sanook.com/news/7722298/,5 เม.ย. 62,"1,925,802",การเมือง_1.txt,การเมือง,text_files/การเมือง_1.txt,[คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาท...,คะแนนเลือกตั้ง 2562: เกาะติดผลเลือกตั้ง วินาที...,คะแนนเลือกตั้ง เกาะติดผลเลือกตั้ง วินาทีตอวิน...,[คะแนนเลือกตั้ง เกาะติดผลเลือกตั้ง วินาทีตอวิ...
1,1,คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต.ค.,https://www.sanook.com/news/1148178/,16 ต.ค. 55,"1,074,542",การเมือง_2.txt,การเมือง,text_files/การเมือง_2.txt,[คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต....,คอหวยตื่น! ป้ายทะเบียนใหม่รถนายกฯ ลุ้นงวด16ต.ค...,คอหวยตืน ป้ายทะเบียนใหมรถนายกฯ ลุ้นงวดต.ค.\nคอ...,"[คอหวยตืน ป้ายทะเบียนใหมรถนายกฯ ลุ้นงวดต.ค., ค..."
2,2,อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร,https://www.sanook.com/news/1596593/,22 พ.ค. 57,"788,755",การเมือง_3.txt,การเมือง,text_files/การเมือง_3.txt,"[อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร, รั...",อัพเดตนาทีต่อนาที! เหตุการณ์หลังรัฐประหาร\nรัฐ...,อัพเดตนาทีตอนาที เหตุการณ์หลังรัฐประหาร\nรัฐปร...,"[อัพเดตนาทีตอนาที เหตุการณ์หลังรัฐประหาร, รัฐป..."
3,3,ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก,https://www.sanook.com/news/1773902/,2 เม.ย. 58,"736,582",การเมือง_4.txt,การเมือง,text_files/การเมือง_4.txt,"[ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก, ห...",ทำความรู้จักกับ มาตรา 44 ใช้แทนกฎอัยการศึก\nหล...,ทำความรู้จักกับ มาตรา ใช้แทนกฎอัยการศึก\nหลัง...,"[ทำความรู้จักกับ มาตรา ใช้แทนกฎอัยการศึก, หลั..."
4,4,ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังปร...,https://www.sanook.com/news/7723326/,25 มี.ค. 62,"706,607",การเมือง_5.txt,การเมือง,text_files/การเมือง_5.txt,[ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังป...,ผลการเลือกตั้ง 2562 นับคะแนนล่าสุด 94% “พลังปร...,ผลการเลือกตั้ง นับคะแนนลาสุด พลังประชารัฐเพื...,[ผลการเลือกตั้ง นับคะแนนลาสุด พลังประชารัฐเพ...


# Finetune

In [7]:
!pip -q install sentence_transformers==2.1.0

     |████████████████████████████████| 78 kB 4.7 MB/s 
     |████████████████████████████████| 4.4 MB 22.0 MB/s 
     |████████████████████████████████| 6.6 MB 48.9 MB/s 
     |████████████████████████████████| 1.2 MB 58.3 MB/s 
     |████████████████████████████████| 101 kB 9.3 MB/s 
     |████████████████████████████████| 596 kB 70.8 MB/s 


In [8]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import itertools

/usr/local/lib/python3.7/dist-packages/huggingface_hub/snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,


In [ ]:
df.head()

In [9]:
model_name = 'mrp/simcse-model-wangchanberta'
model = SentenceTransformer(model_name)
model

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/305 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 32, 'do_lower_case': False}) with Transformer model: CamembertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
min_char = 64

In [11]:
# train_sentences = np.array([x for xs in df['list_clean_text'].values for x in xs])

train_sentences = list(itertools.chain.from_iterable(df['list_clean_text'].values))
len(train_sentences) # 297201

297201

In [12]:
# check sentence > min 
train_sentences = [s for s in train_sentences if len(s) >= min_char]
len(train_sentences)

121790

In [13]:
# Convert train sentences to sentence pairs
train_data = [InputExample(texts=[s, s]) for s in train_sentences]
# DataLoader to batch your data
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)

In [14]:
# Contrastive Loss
train_loss = losses.MultipleNegativesRankingLoss(model) # Contrastive Loss

In [15]:
# test data
!wget https://raw.githubusercontent.com/mrpeerat/Thai-Sentence-Vector-Benchmark/main/sts-test_th.csv

--2022-07-24 09:41:02--  https://raw.githubusercontent.com/mrpeerat/Thai-Sentence-Vector-Benchmark/main/sts-test_th.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 335675 (328K) [text/plain]
Saving to: ‘sts-test_th.csv’

sts-test_th.csv     100%[===================>] 327.81K  --.-KB/s    in 0.02s   

2022-07-24 09:41:02 (14.3 MB/s) - ‘sts-test_th.csv’ saved [335675/335675]



In [16]:
test_data = pd.read_csv('/content/sts-test_th.csv', header=None)
test_data = test_data.dropna().values.tolist()

In [17]:
test_samples = []
for row in test_data:
    score = float(row[4]) / 5.0  # Normalize score to range 0 ... 1
    test_samples.append(InputExample(texts=[row[5], row[6]], label=score)) # 
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=32, name='sts-test')
test_evaluator(model)

0.5412155908136794

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    show_progress_bar=True,
    optimizer_params={'lr': 3e-5},
    output_path='simcse-model-wangchanberta-finetuned-sanook-news',
    save_best_model=True
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1903 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1903 [00:00<?, ?it/s]

In [ ]:
test_evaluator(model)

In [ ]:
from huggingface_hub import notebook_login

notebook_login() 

In [ ]:
model.save_to_hub(repo_name = 'simcse-model-wangchanberta-finetuned-sanook-news')